# Datová akademie, ČSOB, 2023

---

* [Časové řady](#Časové-řady),
    - [úvodní motivace](#Data-a-čas-v-Pythonu),
    - [časové řady](#Časové-datové-typy),
    - [funkce to_datetime](#Funkce-TO_DATETIME),
    - [funkce date_range](#Funkce-DATE_RANGE),
    - [resampling](#Resampling,-snížení-frekvence)
    - [zpřesňování](#Zpřesňování-řady).
* [Výkonné pandy](),
    - [funkce eval](#funkce-EVAL),
    - [výhody eval](#Výkon-EVAL),
    - [funkce query](#Funkce-QUERY),
    - [souhrn k eval a query](#Souhrn-k-EVAL-a-QUERY).
* [Skutečné hodnoty](#Chybějící-hodnoty),
    - [chybějící údaje](#Kolik-mi-chybí-údajů),
    - [encoding](#Encoding),
    - [nekonzistentní data](#Nekonzistentní-data).

<br>

## Časové řady

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.odqbr-09dxxBQjF7x7NyBAHaII%26pid%3DApi&f=1&ipt=5b4a03c1dd38b52108237102ebb350595a5d6ea6a50572f5897ca89fbf5b5445&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Část *frameworku* byla vyvinuta za účelem **finančního modelování**.

Proto je více než dobře vybavena sadou nástrojů, které umí pracovat **s daty, časem a časovými objekty**.

<br>

Uplatnění:
* **Analýza a predikce**: Časové řady umožňují analyzovat vývoj hodnot v čase a předpovídat budoucí trendy nebo sezónní změny,

* **Efektivní manipulace s časem**: Pandas poskytuje nástroje pro efektivní práci s časovými řadami, což zjednodušuje úkoly, jako je agregace, interpolace nebo časové posuny,

* **Časově závislé analýzy**: Časové řady umožňují provádět časově závislé analýzy, jako je detekce změn v trendech, identifikace sezónních vlivů nebo identifikace časových závislostí mezi proměnnými,

* **Flexibilní indexace**: Pandas podporuje flexibilní indexaci časových řad, což usnadňuje filtrování, řazení a výběr dat na základě časových značek nebo rozsahů,

* **Kompatibilita s dalšími knihovnami**: Pandas je kompatibilní s mnoha dalšími knihovnami pro analýzu časových řad, jako je `statsmodels` nebo `scikit-learn`.
    
<br>

Jde například o údaje typu:
* *timestampy*, údaj odkazující na **konkrétní časový okamžik** (datum a čas, např. `4. července 2015 v 7:00 hod.`),
* *časové intervaly*, tedy období odkazují na **délku času mezi konkrétním začátkem a koncem** (např. intervaly ze dne na den),
* *time delta* objekty, tedy **přesné délky času** (např. `22,22` sekundy).

<br>

#### Data a čas v Pythonu

---

Standardní výbavou Pythonu jsou knihovny `datetime`:

In [87]:
from datetime import datetime

In [88]:
datetime?

Init signature: datetime(self, /, *args, **kwargs)
Docstring:     
datetime(year, month, day[, hour[, minute[, second[, microsecond[,tzinfo]]]]])

The year, month and day arguments are required. tzinfo may be None, or an
instance of a tzinfo subclass. The remaining arguments may be ints.
File:           /usr/lib/python3.8/datetime.py
Type:           type
Subclasses:     ABCTimestamp, _NaT

In [ ]:
datetime(year=2023, month=4, day=5)

<br>

Nebo knihovna `dateutil` pro *parsování* datových typů z různých **stringových zadání**:

In [89]:
from dateutil import parser

In [90]:
parser?

Type:        module
String form: <module 'dateutil.parser' from '/home/matous/projects/data-csob-2023/shared/onsite/env/lib/python3.8/site-packages/dateutil/parser/__init__.py'>
File:        ~/projects/data-csob-2023/shared/onsite/env/lib/python3.8/site-packages/dateutil/parser/__init__.py
Docstring:  
This module offers a generic date/time string parser which is able to parse
most known formats to represent a date and/or time.

This module attempts to be forgiving with regards to unlikely input formats,
returning a datetime object even for dates which are ambiguous. If an element
of a date/time stamp is omitted, the following rules are applied:

- If AM or PM is left unspecified, a 24-hour clock is assumed, however, an hour
  on a 12-hour clock (``0 <= hour <= 12``) *must* be specified if AM or PM is
  specified.
- If a time zone is omitted, a timezone-naive datetime is returned.

If any other elements are missing, they are taken from the
:class:`datetime.datetime` object passed to th

In [ ]:
date = parser.parse("5th of april, 2023")

In [ ]:
date

<br>

Kde pomocí metody `strftime` můžeš vypsat den:

In [ ]:
date.strftime("%A")

<br>

#### Data a čas v numpy

---

Některé nedostatky uvnitř knihoven `datetime` a `dateutil` vedli ke vzniku sady nástrojů.

Tyto doplňky vznikly pod hlavičkou knihovny `numpy`.

In [ ]:
from numpy import array, arange

In [ ]:
date = array('2023-04-05', dtype=np.datetime64)

In [ ]:
date

<br>

Pokud potřebuješ pole následujících 7 dní:

In [ ]:
date + arange(7)

<br>

Vzhledem k jednotnému datovu typu v poli pro **numpy** `datetime64` může tento typ operace
provádět mnohem rychleji, než přímo v Pythonu `datetime` objekty, zejména když objekty nabývají na velikosti.

#### Data a čas v pandách

---

Jde o kombinace objektů z obou předchozích podkapitol.

Ty dávají dohromady to nejlepší prostředky pro zacházení s časem.

In [ ]:
from pandas import to_datetime

In [ ]:
date = to_datetime("5th of April, 2023")

In [ ]:
date

In [ ]:
date.strftime("%A")

<br>

### Základní objekty

---

V podstatě jde o hlavní nástroj, který tato knihovna dovede nabídnout.

Tedy práce s univerzálními datovými typy pro čas.

Mezi základní objekty pro práci s časem patří:
* `Timestamp`, typ (související struktura Indexu `DatetimeIndex`,
* `Period`, typ (.. `PeriodIndex`),
* `Timedelta`, typ (.. `TimedeltaIndex`).

#### Timestamp

---

Základní datový typ do časových řad.

Přiřazuje pandasí hodnoty k skutečným konkrétním časovým bodům.

In [106]:
from pandas import Timestamp

In [110]:
Timestamp("2023-05-04")

Timestamp('2023-05-04 00:00:00')

In [111]:
Timestamp(2023, 5, 4)

Timestamp('2023-05-04 00:00:00')

In [118]:
type(Timestamp(2023, 5, 4))

pandas._libs.tslibs.timestamps.Timestamp

<br>

#### Period

---

V mnoha případech je však přirozenější spojit proměnné, jako je změna, s časovým rozpětím.

`Period` představuje časový interval nebo období, které má začátek a konec.

In [112]:
from pandas import Period

In [114]:
Period("2023-05", freq="D")

Period('2023-05-01', 'D')

In [119]:
type(Period("2023-05", freq="D"))

pandas._libs.tslibs.period.Period

Hlavním rozdílem pro `Timestamp` a `Period` je v rámci jejich zapracování.

`Timestamp` pracuje s konkrétním časovým okamžikem.

`Period` naopak s časovým intervalem.

<br>

#### Indexování časem

---

`DatetimeIndex` obsahuje časové značky (*timestamp*), které jsou uloženy ve formátu `datetime64` s nanosekundovou přesností.

Tento objekt umožňuje efektivní práci s časovými řadami a poskytuje mnoho funkcí pro manipulaci s daty a časy.

Výhody práce s `DatetimeIndex` objektem:
* Časové zóny,
* frekvence,
* časově závislé selekce,
* atributy časových značek,
* operace s časem.

In [120]:
from pandas import DatetimeIndex, Series

In [121]:
datumy = ["2023-04-05", "2022-04-05", "2021-04-05", "2020-04-05"]

In [122]:
indexy = DatetimeIndex(datumy)

In [123]:
hodnoty = [to_datetime(den).strftime("%A") for den in datumy]

In [124]:
df_hodnoty = Series(hodnoty, index=indexy)

In [125]:
df_hodnoty

2023-04-05    Wednesday
2022-04-05      Tuesday
2021-04-05       Monday
2020-04-05       Sunday
dtype: object

In [126]:
type(indexy)

pandas.core.indexes.datetimes.DatetimeIndex

In [127]:
df_hodnoty["2020": "2022"]

/tmp/ipykernel_7941/3590313258.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df_hodnoty["2020": "2022"]


2022-04-05    Tuesday
2021-04-05     Monday
2020-04-05     Sunday
dtype: object

<br>

### Funkce TO_DATETIME

---

Nejčastější datové typy, které lze zapsat přímo, ovšem běžnější je pracovat s funkcí `to_datetime`.

Funkce `to_datetime` umí parsovat různé stringové formáty.

In [128]:
from pandas import to_datetime

<br>

#### Práce s jedním datumem

---

In [129]:
datum = to_datetime("05/04/2023")

In [130]:
datum

Timestamp('2023-05-04 00:00:00')

In [131]:
type(datum)

pandas._libs.tslibs.timestamps.Timestamp

Pokud do funkce `to_datetime` vložíš jedinou hodnotu, vrací objekty typu `Timestamp`.

<br>

#### Specifický formát

---

Pokud se parser ve funkci `to_datetime` ztratí nebo tvoje zadání neodpovídá jeho vyhotovení, můžeš zadat formát explicitně:

In [133]:
vystup = to_datetime("12-11-2010 00:00", format="%d-%m-%Y %H:%M")

In [137]:
vystup.day

12

In [135]:
vystup.month

11

In [136]:
vystup.year

2010

<br>

#### Práce s několika datumy

---

Stejně jak umí funkce *parsovat* jednu hodnotu, umí zpracovat pole časových stringů:

In [138]:
datumy = to_datetime([
    datetime(2023, 4, 5), "5th of April 2023", "2023-Apr-5", "05-04-2023", "20230405"]
)

In [139]:
datumy

DatetimeIndex(['2023-04-05', '2023-04-05', '2023-04-05', '2023-05-04',
               '2023-04-05'],
              dtype='datetime64[ns]', freq=None)

In [140]:
type(datumy)

pandas.core.indexes.datetimes.DatetimeIndex

Zatímco pole hodnot s datumy, které funkce `to_datetime` zpracuje vrací objekt typu `DatetimeIndex`.

<br>

Objekt `DatetimeIndex` potom můžeš konvertovat na `PeriodIndex` pomocí *metody* `to_period`:

In [141]:
datumy.to_period?

Signature: datumy.to_period(*args, **kwargs)
Docstring:
Cast to PeriodArray/Index at a particular frequency.

Converts DatetimeArray/Index to PeriodArray/Index.

Parameters
----------
freq : str or Offset, optional
    One of pandas' :ref:`offset strings <timeseries.offset_aliases>`
    or an Offset object. Will be inferred by default.

Returns
-------
PeriodArray/Index

Raises
------
ValueError
    When converting a DatetimeArray/Index with non-regular values,
    so that a frequency cannot be inferred.

See Also
--------
PeriodIndex: Immutable ndarray holding ordinal values.
DatetimeIndex.to_pydatetime: Return DatetimeIndex as object.

Examples
--------
>>> df = pd.DataFrame({"y": [1, 2, 3]},
...                   index=pd.to_datetime(["2000-03-31 00:00:00",
...                                         "2000-05-31 00:00:00",
...                                         "2000-08-31 00:00:00"]))
>>> df.index.to_period("M")
PeriodIndex(['2000-03', '2000-05', '2000-08'],
            dtype=

<br>

Tato metoda je užitečná, pokud chcete převést časové řady **na určité časové období**.

Když budeš třeba potřebovat převést hodnoty **z denních dat na měsíční data**.

#### Denní data

In [ ]:
datumy.to_period("D")

#### Měsíční data

In [ ]:
datumy.to_period("M")

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', periods=10, freq='D'),
    'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
df_period = df.to_period(freq='Y')

<br>

#### Časové zony

---


In [101]:
tz_datum = datum.tz_localize("UTC")

In [102]:
tz_datum

Timestamp('2023-05-04 00:00:00+0000', tz='UTC')

In [103]:
nove_tz_datum = tz_datum.tz_convert("US/Pacific")

In [104]:
nove_tz_datum

Timestamp('2023-05-03 17:00:00-0700', tz='US/Pacific')

<br>

### TimedeltaIndex

---

Jde o rozdíl v jednotkách času.

Nejčastěji se s tímto objektem setkáš, pokud potřebuješ získat **rozdíl mezi dvěma datumy**.

In [ ]:
specialni_datumy = to_datetime([
    datetime(2022, 4, 5), "5th of April 2021", "2020-Apr-5", "05-04-2019", "20180405"]
)

In [ ]:
datumy - specialni_datumy

<br>

### Funkce DATE_RANGE

---

Aby bylo zadání řady (sekvence) dat pohodlnější, vyzkoušej funkci `date_range`.

Obdobně potom pracují související funkce:
* `date_range`, timestampy,
* `period_range`, periody,
* `timedelta_range`, pro delty.

In [143]:
from pandas import date_range

<br>

#### Počet period

---

Nachystá objekt, který vypracuje tolik period, které definuješ pomocí parametru `periods`:

In [144]:
datumy_ind = date_range("01-01-1992", periods=8)

In [145]:
datumy_ind

DatetimeIndex(['1992-01-01', '1992-01-02', '1992-01-03', '1992-01-04',
               '1992-01-05', '1992-01-06', '1992-01-07', '1992-01-08'],
              dtype='datetime64[ns]', freq='D')

<br>

#### Frekvence

---

In [146]:
datumy_mesicne_ind = date_range("01-01-1992", periods=8, freq="M")

In [147]:
datumy_mesicne_ind

DatetimeIndex(['1992-01-31', '1992-02-29', '1992-03-31', '1992-04-30',
               '1992-05-31', '1992-06-30', '1992-07-31', '1992-08-31'],
              dtype='datetime64[ns]', freq='M')

<br>

#### Hodinové periody

---

In [148]:
from pandas import timedelta_range

In [149]:
hodinove_ind = timedelta_range(0, periods=12, freq="H")

In [150]:
hodinove_ind

TimedeltaIndex(['0 days 00:00:00', '0 days 01:00:00', '0 days 02:00:00',
                '0 days 03:00:00', '0 days 04:00:00', '0 days 05:00:00',
                '0 days 06:00:00', '0 days 07:00:00', '0 days 08:00:00',
                '0 days 09:00:00', '0 days 10:00:00', '0 days 11:00:00'],
               dtype='timedelta64[ns]', freq='H')

<br>

### Frekvence

Časovou paletou frekvencí, kterou framework `pandas` nabízí je tato tabulka:

| String | Popisek |
| :-: | :- |
| `D` | kalendářní den |
| `W` | týden |
| `M` | konec měsíce |
| `Q` | konec čtvrtletí |
| `A` | konec roku |
| `H` | hodiny |
| `T` | minuty |
| `S` | vteřiny |
| `B` | pracovní den |
| `BM` | konec pracovníḧo měsíce |
| `BQ` | konec pracovního čtvrtletí |

#### Netradiční frekvence

---

In [151]:
timedelta_range(0, periods=5, freq="1H15T")

TimedeltaIndex(['0 days 00:00:00', '0 days 01:15:00', '0 days 02:30:00',
                '0 days 03:45:00', '0 days 05:00:00'],
               dtype='timedelta64[ns]', freq='75T')

<br>

### Resampling, snížení frekvence

---

*Resampling* nebo také *převzorkování* je proces, který upravuje frekvenci časové řady.

*Resampling* se obvykle používá **pro snížení frekvence** (z hodinových dat na denní data).

*Resampling* zahrnuje *agregaci* dat.

V `pandas` se k tomu používá metoda `resample`, která má jako parametr novou frekvenci, na kterou chcete data převést:

In [ ]:
import pandas as pd

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', periods=60, freq='D'),
    'sales': range(60)
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
monthly_data = df.resample('M').sum()

In [ ]:
print(monthly_data)

<br>

### Zpřesňování řady

---

Zahrnuje **zvýšení frekvence časových řad**.

Přičemž se nově vytvořené hodnoty obvykle interpolují nebo doplňují nějakou konstantou.

V pandas se k zpřesňování používá metoda `asfreq`, která má jako parametr novou frekvenci, na kterou chcete data převést.

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', end='2022-03-01', freq='MS'),
    'sales': [100, 120, 150]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
daily_data = df.asfreq('D', method='ffill')

In [ ]:
daily_data.head(20)

Metoda `asfreq` bere jako parametr novou frekvenci, na kterou chceme data převést.

V případě výše jde o frekvenci `D`, což znamená **denní data**.

Protože při zpřesňování se **vytvoří chybějící hodnoty** (v důsledku zvýšení frekvence), je potřeba zvolit metodu pro jejich doplnění.

Výš jde o metodu `forward fill` (parametr `method='ffill'`), která kopíruje předchozí hodnotu pro výplň chybějících hodnot.

Další možností je použít metodu `'back fill'` (parametr `method='bfill'`), která kopíruje následující hodnotu pro výplň chybějících hodnot.

<br>

**🧠 CVIČENÍ 🧠, procvič si časové řady**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Celkový prodej za každý měsíc.
2. Průměrný prodej za každý den v týdnu.
3. Denní prodej za poslední týden.

In [ ]:
import pandas as pd
import numpy as np

denni_datumy = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')
prodeje = np.random.randint(10, 100, size=(len(date_rng),))

data = {
    'datumy': denni_datumy,
    'prodeje': prodeje
}

df_prodeje = pd.DataFrame(data)
df_prodeje.set_index('datumy', inplace=True)

In [ ]:
df_prodeje

### Resampling

---

*Resampling* nebo také *převzorkování* je proces, který upravuje frekvenci časové řady.

*Resampling* se obvykle používá **pro snížení frekvence** (z hodinových dat na denní data).

*Resampling* zahrnuje *agregaci* dat.

V `pandas` se k tomu používá metoda `resample`, která má jako parametr novou frekvenci, na kterou chcete data převést:

In [ ]:
import pandas as pd

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', periods=60, freq='D'),
    'sales': range(60)
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
monthly_data = df.resample('M').sum()

In [ ]:
print(monthly_data)

<br>

### Zpřesňování

---

Zahrnuje **zvýšení frekvence časových řad**.

Přičemž se nově vytvořené hodnoty obvykle interpolují nebo doplňují nějakou konstantou.

V pandas se k zpřesňování používá metoda `asfreq`, která má jako parametr novou frekvenci, na kterou chcete data převést.

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', end='2022-03-01', freq='MS'),
    'sales': [100, 120, 150]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
daily_data = df.asfreq('D', method='ffill')

In [ ]:
daily_data.head(20)

Metoda `asfreq` bere jako parametr novou frekvenci, na kterou chceme data převést.

V případě výše jde o frekvenci `D`, což znamená **denní data**.

Protože při zpřesňování se **vytvoří chybějící hodnoty** (v důsledku zvýšení frekvence), je potřeba zvolit metodu pro jejich doplnění.

Výš jde o metodu `forward fill` (parametr `method='ffill'`), která kopíruje předchozí hodnotu pro výplň chybějících hodnot.

Další možností je použít metodu `'back fill'` (parametr `method='bfill'`), která kopíruje následující hodnotu pro výplň chybějících hodnot.

<br>

**🧠 CVIČENÍ 🧠, procvič si časové řady**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Celkový prodej za každý měsíc.
2. Průměrný prodej za každý den v týdnu.
3. Denní prodej za poslední týden.

In [133]:
from numpy.random import randint
from pandas import date_range, DataFrame

denni_datumy = date_range(start='2022-01-01', end='2022-12-31', freq='D')
prodeje = randint(10, 100, size=(len(denni_datumy),))

data = {
    'datumy': denni_datumy,
    'prodeje': prodeje
}

df_prodeje = DataFrame(data)
df_prodeje.set_index('datumy', inplace=True)

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    mesicni_prodej = df_prodeje.resample('M').sum()

    df_prodeje['weekday'] = df_prodeje.index.weekday
    denni_prumer = df_prodeje.groupby('weekday')['prodeje'].mean()

    prodej_posledni_vikend = df_prodeje.loc['2022-12-25':, 'prodeje']
    ```
</details>

* [High performance](),
    - [úvodní motivace](),
    - [eval](),
    - [query](),
    - [caveats]().

## Vysoký výkon v PANDAS

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.riOR-y2_yu_2cGdEN2e6-wAAAA%26pid%3DApi&f=1&ipt=638c4516d155ff9768066e17887a621cbad0de6bce639a40be98f5dedbce67ac&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Z předchozích ukázek a popisků (seskupování) tedy můžeš chápat, že skutečný výkon *frameworku* tkví v převedení **základních operací do jazyka C**.

Tyto abstrakce jsou účinné a efektivní pro mnoho běžných případů použití.

Vytvářejí ale spousty dočasných objektů, což může způsobit nepřiměřenou režii výpočetního času a paměti.

### EVAL a QUERY

---

Přímý přístup k operacím rychlosti C bez nákladné alokace `pandas` poskytuje funkce `eval` a `query`.

Ty spolehájí na funkci `numexpr`.

<br>

### funkce EVAL

---

Dva důvody, proč používat funkci `eval` místo obyčejného Pythonu:
1. velké objekty typu `DataFrame` jsou vyhodnocovány efektivněji,
2. aritmetické a logické výrazy projdou naráz základním enginem (pomocí `numexpr`).

**Pozor!** Dobrá obecná pomůcka říká, že není nutné využívat funkci `eval` pro tabulky, které jsou kratší než 10 000 řádků. 

### Použití EVAL

---



Funkce `eval` slouží k efektivnímu vyhodnocení **aritmetických výrazů** na:
* objekty `DataFrame`,
* objekty `Series`.

Její největší výhodou je **rychlost a efektivita**, která spočívá v minimalizaci paměťové zátěže a zrychlení operací.

Když pracuješ **s velkými datovými sadami**, paměťová zátěž a rychlost se stávají klíčovými faktory.

Funkce `eval` optimalizuje výpočetní proces tím, že snižuje množství alokované paměti a zrychluje výpočetní operace.

Ukázka použití funkce `eval`:

### Ukázka EVAL

---

Pro zadaný *dummy dataset* vytvoř **nový sloupeček** `sloupec_D`, podle vzoru:
`A + B * C`

In [1]:
from pandas import DataFrame, eval

In [2]:
data = {'A': [1, 2, 3, 4, 5],
        'B': [6, 7, 8, 9, 10],
        'C': [11, 12, 13, 14, 15]}

In [3]:
df_data = DataFrame(data)

In [5]:
df_data

,A,B,C
0,1,6,11
1,2,7,12
2,3,8,13
3,4,9,14
4,5,10,15


In [9]:
df_data['sloupec_D'] = df_data['A'] + df_data['B'] * df_data['C']

In [10]:
df_data

,A,B,C,sloupec_D
0,1,6,11,67
1,2,7,12,86
2,3,8,13,107
3,4,9,14,130
4,5,10,15,155


<br>

*Alternativní způsob*, jak provést stejný výpočet, pokud máš dlouhou tabulku nebo sloupeček, je použití funkce `eval`:

In [11]:
df_data['sloupecek_D'] = df_data.eval('A + B * C')

<br>

Funkce `eval` zpracovává zadaný `str` podobně jako built-in funkce `eval`.

In [12]:
df_data

,A,B,C,sloupec_D,sloupecek_D
0,1,6,11,67,67
1,2,7,12,86,86
2,3,8,13,107,107
3,4,9,14,130,130
4,5,10,15,155,155


Souhrnně lze říci:
* funkci `eval` použij tehdy, pokud je dataset dlouhý,
* ve všech ostatních scénářích, kde nepotřebuješ šetřit pamět a čas použij klasickou syntaxi.

### Výkon EVAL

---

In [22]:
import numpy as np
import pandas as pd

In [14]:
pocet_radku, pocet_sloupcu = 100_000, 100

In [15]:
rng = np.random.RandomState(42)

In [29]:
dframe_1, dframe_2, dframe_3, dframe_4 = [DataFrame(rng.rand(pocet_radku, pocet_sloupcu)) for _ in range(4)]

In [30]:
%timeit dframe_1 + dframe_2 + dframe_3 + dframe_4

523 ms ± 69.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit pd.eval("dframe_1 + dframe_2 + dframe_3 + dframe_4")

354 ms ± 64.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<br>

### Funkce QUERY

---

Funkce `query` slouží k efektivnímu **filtrování řádků**.

Filtrování probíhá na základě **zadané podmínky**.


Hlavní výhodou jsou:
* **snadná čitelnost**, umožňuje zapisovat podmínky ve snadno čitelné formě, zejména v případě složitějších dotazů,
* **rychlost**, může být rychlejší než tradiční metody filtrování,(zejména pro velké datové sady).

### Ukázka QUERY

---

Pro zadaný *dummy dataset* vyfiltruj takové záznamy, kde:
1. Hodnoty ze sloupce A budou **menší než 4**,
2. hodnoty ze sloupce B budou **větší než 6**,
3. hodnoty ze sloupce C budou **větší než 11**.

In [32]:
data = {'A': [1, 2, 3, 4, 5],
        'B': [6, 7, 8, 9, 10],
        'C': [11, 12, 13, 14, 15]}

In [33]:
df_data = DataFrame(data)

In [34]:
df_data

,A,B,C
0,1,6,11
1,2,7,12
2,3,8,13
3,4,9,14
4,5,10,15


In [39]:
df_filtrovane = df_data[(df_data['A'] < 4) & (df_data['B'] > 6) & (df_data['C'] > 11)]

In [40]:
df_filtrovane

,A,B,C
1,2,7,12
2,3,8,13


Zapisování pomocí *boolean indexování* je ovšem náročné jak pro zápis, tak pro čtení.

<br>

Pomocí funkce `query`:

In [37]:
df_filtrovane = df_data.query('A < 4 and B > 6 and C > 11')

In [38]:
df_filtrovane

,A,B,C
1,2,7,12
2,3,8,13


### Nedostatky QUERY

---

1. **Omezená syntaxe**, má omezenější syntaxi ve srovnání s běžným zápisem v Pythonu a nemusí podporovat všechny operace,
2. **Názvy sloupců**, pokud názvy sloupců obsahují **mezery nebo znaky**, které nejsou platnými identifikátory Pythonu, musíš je uvést v závorkách a použít symbol `@` pro reference na proměnné mimo `DataFrame`.

In [47]:
df_cisla = pd.DataFrame({'Sloupec 1': [1, 2, 3, 4, 5], 'Sloupec 2': [10, 20, 30, 40, 50]})

In [48]:
vystup_1 = df_cisla.query('Sloupec 1 < 3')

SyntaxError: invalid syntax (<unknown>, line 1)

In [49]:
vystup_2 = df_cisla.query('`Sloupec 1` < 3')

In [50]:
vystup_2

,Sloupec 1,Sloupec 2
0,1,10
1,2,20


### Souhrn k EVAL a QUERY

---

Hlavním účelem těchto funkcí je aplikace na skutečně **velké datové sety**.

Rozdíly u drobnějších datasetů **jsou často minimální**.

Velký vliv na výstupný čas dělá parametr `engine`, který se snaží pracovat s knihovnou `numexpr`.

<br>

**🧠 CVIČENÍ 🧠, procvič si funkce EVAL a QUERY**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Vyber studenty s věkem **mezi 20 a 22 lety (včetně)** a email, který obsahuje string **gmail.com**,
2. vypočtěte *BMI* (Body Mass Index) pro každého studenta a přidejte ho do nového sloupce (vzorec: *váha[kg] / (výška[m] * výška[m])*)
3. vyberte studenty s BMI vyšším než 22 a kteří pochází z měst Brno a Praha.

In [88]:
uzivatele = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [19, 20, 21, 22, 23],
    'Height_cm': [160, 170, 175, 180, 165],
    'Weight_kg': [50, 65, 70, 80, 55],
    'Email': ["alice@gmail.com", "bob@tech-users.com", "charlie91@mail.com", "david_bB@gmail.com", "eevee@my_domain.net"],
    'Address': [
        "Vrbová 215/14, 602 00, Brno, Česká republika",
        "Jilmová 862, 400 11, Ústí nad Labem, Česká republika",
        "Kamenická 37/2, 170 00, Praha, Česká republika",
        "Podlipná 1284/6, 460 01, Liberec, Česká republika",
        "Havlíčkova 1703, 500 02, Hradec Králové, Česká republika"
    ]
}

In [89]:
df_uzivatele = DataFrame(uzivatele)

In [90]:
df_uzivatele

,Name,Age,Height_cm,Weight_kg,Email,Address
0,Alice,19,160,50,alice@gmail.com,"Vrbová 215/14, 602 00, Brno, Česká republika"
1,Bob,20,170,65,bob@tech-users.com,"Jilmová 862, 400 11, Ústí nad Labem, Česká rep..."
2,Charlie,21,175,70,charlie91@mail.com,"Kamenická 37/2, 170 00, Praha, Česká republika"
3,David,22,180,80,david_bB@gmail.com,"Podlipná 1284/6, 460 01, Liberec, Česká republika"
4,Eve,23,165,55,eevee@my_domain.net,"Havlíčkova 1703, 500 02, Hradec Králové, Česká..."


<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    vyber_uzivatel = df_uzivatele.query("20 <= Age <= 22 and Email.str.contains('gmail.com')")

    df_uzivatele['Height_m'] = df_uzivatele.eval('Height_cm / 100')
    df_uzivatele['BMI'] = df_uzivatele.eval('Weight_kg / (Height_m * Height_m)').round(1)
    
    df_prodeje['weekday'] = df_prodeje.index.weekday
    average_weekday_sales = df_prodeje.groupby('weekday')['prodeje'].mean()
    
    df_uzivatele["City"] = df_uzivatele["Address"].apply(lambda x: x.split(",")[2])
    students_with_high_bmi = df_uzivatele.query('BMI > 22 and City in ["Brno", "Praha"]')
    ```
</details>

<br>

## Chybějící hodnoty

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.JJu86kKRph1LHt5M7agODQHaHa%26pid%3DApi&f=1&ipt=5cfb9e9f7b451b3a4f7a3ebdc4eb99177d477f20f8145cd818cc4966143f8b2b&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Čištění dat je proces, který patří k samotné práci s daty.

Spolu s některými souvisejícími úkony, patří mezi ty více frustrující.

Nesmyslné datové typy, zkomolené časové údaje, nefungující transformace.

In [116]:
df_uzivatele = DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [None, 20, None, 22, 23],
    'Height_cm': [160, 170, 175, None, 165],
    'Weight_kg': [50, 65, 70, 80, 55]
})

In [117]:
df_uzivatele.head(3)

,Name,Age,Height_cm,Weight_kg
0,Alice,NaN,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,NaN,175.0,70


Ihned po načtení je nejlepší data prozkoumat.

Nejenom datové typy ale také mít povědomí **o chybějících hodnotách**:

In [107]:
df_uzivatele.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       5 non-null      object 
 1   Age        3 non-null      float64
 2   Height_cm  4 non-null      float64
 3   Weight_kg  5 non-null      int64  
 4   Email      4 non-null      object 
dtypes: float64(2), int64(1), object(2)
memory usage: 328.0+ bytes


<br>

Pomocí metody `info` si uděláš aspoň povrchní přehled o hodnotách, datových typech a chybějících hodnotách.

<br>

### Kolik mi chybí údajů

---

Vždycky přistupuj k datům s mírnou skepsí.

Otázkou potom není, *JESTLI* mi chybí data, ale *KOLIK* dat mi chybí **a kde**.

In [108]:
chybejici_hodnoty = df_uzivatele.isnull()

In [109]:
chybejici_hodnoty

,Name,Age,Height_cm,Weight_kg,Email
0,False,True,False,False,False
1,False,False,False,False,False
2,False,True,False,False,False
3,False,False,True,False,False
4,False,False,False,False,True


<br>

Pomocí metody `isnull` nahradíš všechny chybějící hodnoty v tabulce/sloupcích pomocí `True`.

Pokud nechybí, nahradí s `False`.

In [110]:
type(chybejici_hodnoty)

pandas.core.frame.DataFrame

Takový výstup ale není zcela reprezentativní.

Proto je potřeba, sečíst všechny `True` hodnoty po sloupcích:

In [112]:
chybejici_hodnoty = df_uzivatele.isnull().sum()

In [113]:
chybejici_hodnoty

Name         0
Age          2
Height_cm    1
Weight_kg    0
Email        1
dtype: int64

<br>

Opět v jednotkách to nemusí zcela prozrazovat chybovost.

Nejlepším indikátorem ale bývají procenta z celkového množství hodnot ve sloupci:

In [122]:
celkem_zaznamu = df_uzivatele.shape[0]  # 0 ... Indexy, 1 ... sloupce

In [123]:
celkem_chybi = chybejici_hodnoty

In [124]:
v_procentech = round(celkem_chybi / celkem_zaznamu * 100, 1)

In [125]:
v_procentech

Name          0.0
Age          40.0
Height_cm    20.0
Weight_kg     0.0
Email        20.0
dtype: float64

<br>

případně celkové množství záznamů ve všech sloupcích:

In [126]:
vsechny_bunky = df_uzivatele.shape[0] * df_uzivatele.shape[1]  # 5 * 4

In [127]:
vsechny_chybejici_hodnoty = chybejici_hodnoty.sum()            # 4

In [128]:
celkem_v_procentech = round(vsechny_chybejici_hodnoty / vsechny_bunky * 100, 1)

In [129]:
celkem_v_procentech

20.0

<br>

V této ukázce tedy chybí 20 % dat a to není málo!

<br>

### Co s chybějícími hodnotami

---

Je důležité vědět, proč máš takové množství chybějících hodnot.

Chybí ti data, protože neexistují, nebo protože nejsou součástí záznamů v datasetu.

V takových krocích je obvykle nutné, pochopit podstatu údajů (dokumentace, specifikace,..).

Nejjednodušší způsoby, jak s chybějícími daty naložit (ale ne efektivní!!):
1. Zahodit chybějící hodnoty,
2. doplnit chybějící hodnoty.

#### Zahodit chybějící hodnoty

In [130]:
df_uzivatele.dropna()

,Name,Age,Height_cm,Weight_kg
1,Bob,20.0,170.0,65
4,Eve,23.0,165.0,55


<br>

V tento okamžik si zahodíš všechny záznamy, které **postrádaly nějakou hodnotu**.

Někdy je jednodušší zahodit sloupeček, který má chybějící hodnotu:

In [131]:
df_uzivatele.dropna(axis=1)

,Name,Weight_kg
0,Alice,50
1,Bob,65
2,Charlie,70
3,David,80
4,Eve,55


#### Doplnit automaticky hodnoty

In [ ]:
df_uzivatele.fillna(0)

<br>

Pomocí funkce `fillna` můžeš vyplnit místo chybějících hodnot předdefinovanou vlastní hodnotu.

Nebo pomocí argumentu pro `method` pracovat s elegantnějším zadáním.

Nahradit chybějící hodnoty hodnotou, která následuje bezprostředně za ní ve stejném sloupci (To má velký smysl u souborů dat, kde mají pozorování nějaké logické pořadí.)

In [144]:
df_uzivatele.fillna(method='bfill', axis=0).fillna(0)

,Name,Age,Height_cm,Weight_kg
0,Alice,20.0,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,22.0,175.0,70
3,David,22.0,165.0,80
4,Eve,23.0,165.0,55


## Encoding

---

Problémy s kódováním jsou běžné.

Jde o proces, který mapuje bajtové stringy (0110101011) na uživatelsky čitelný text ("ahoj!").

Tento problém nastane, pokud se snažíš načíst zdroj v jiné kódovací sadě, než byl soubor zapsaný.

<br>

Protože je těchto sad hodně, občas skončíš s escapovanými znaky, nebo s neznámými kliky-háky:

In [ ]:
æ–‡å

<br>

Případně pokud nelze domapovat znaky:

��

<br>

V Pythonu 2 nebylo lehké encoding zajistit.

V Pythonu 3 je celý proces o dost jednodušší.

Standardem pro práci bývá obyčejně UTF-8. Na to ale zrovna v našich končinách nemůžeš spoléhat.

In [1]:
veta = "Matouš zaplatil 100 $"

In [2]:
veta?

Type:        str
String form: Matouš zaplatil 100 $
Length:      21
Docstring:  
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.

<br>

Údaj, který vidíš výš je datový typ `str`.

In [3]:
prevedena_veta = veta.encode("utf-8", errors="replace")

In [4]:
prevedena_veta?

Type:        bytes
String form: b'Matou\xc5\xa1 zaplatil 100 $'
Length:      22
Docstring:  
bytes(iterable_of_ints) -> bytes
bytes(string, encoding[, errors]) -> bytes
bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer
bytes(int) -> bytes object of size given by the parameter initialized with null bytes
bytes() -> empty bytes object

Construct an immutable array of bytes from:
  - an iterable yielding integers in range(256)
  - a text string encoded using the specified encoding
  - any object implementing the buffer API.
  - an integer

<br>

Dál je možné, převést `str` na `bytes`. Tedy sekvenci čísel.

Jak ale teď tato sekvence vypadá:

In [6]:
prevedena_veta

b'Matou\xc5\xa1 zaplatil 100 $'

<br>

Celou větu můžeš zpátky kódovat pomocí funkce `decode` z `bytes` na `str`:

In [8]:
print(prevedena_veta.decode("utf-8"))

Matouš zaplatil 100 $


<br>

Pokud ale zvolíš jinou sadu, nemusíš dostat stejné hodnoty:

In [10]:
print(prevedena_veta.decode("windows-1250"))

MatouĹˇ zaplatil 100 $


In [13]:
print(prevedena_veta.decode("ascii"))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 5: ordinal not in range(128)

<br>

Toto a další neúspěšné postupy je samozřejmě špatně a ty se tomu chceš určitě vyhnout.

In [11]:
from pandas import read_csv

In [20]:
chybna_sada = read_csv("neznamy_vzorek.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 7955: invalid start byte

<br>

Ne vždy je možné, defaultně pracovat se znakovou sadou UTF-8.

Ve výstupu je vidět, že jde o směs několika sad a ty musíš vyzkoušet, jaká bude fungovat.

In [14]:
import sys

In [15]:
sys.getdefaultencoding()

'utf-8'

In [41]:
!pip install chardet

     |████████████████████████████████| 199 kB 1.8 MB/s eta 0:00:01


In [42]:
from chardet.universaldetector import UniversalDetector

In [43]:
detector = UniversalDetector()

In [45]:
for line in open("neznamy_vzorek.csv", 'rb'):
    detector.feed(line)
    if detector.done:
        break

detector.close()
print(detector.result)

{'encoding': 'Windows-1252', 'confidence': 0.7299788775910958, 'language': ''}


<br>

Pomocí knihovny `chardet` / `charset_normalizer` identifikuješ, o jaké znakové sady jde:

Nyní máš lepší představu o tom, jakou znakovou sadu soubor používá.

Opatrně na délku `str`, který detekuješ.

Pokud je příliš krátký, může to ovlivnit výsledek.

Naopak pokud je příliš dlouhý, může trvat jeho načtení.

In [46]:
spravna_sada = read_csv("neznamy_vzorek.csv", encoding="Windows-1252")

/tmp/ipykernel_7941/3584548271.py:1: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  spravna_sada = read_csv("neznamy_vzorek.csv", encoding="Windows-1252")


In [47]:
spravna_sada.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


In [48]:
spravna_sada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323750 entries, 0 to 323749
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              323750 non-null  int64  
 1   name            323746 non-null  object 
 2   category        323745 non-null  object 
 3   main_category   323750 non-null  object 
 4   currency        323750 non-null  object 
 5   deadline        323750 non-null  object 
 6   goal            323750 non-null  object 
 7   launched        323750 non-null  object 
 8   pledged         323750 non-null  object 
 9   state           323750 non-null  object 
 10  backers         323750 non-null  object 
 11  country         323750 non-null  object 
 12  usd pledged     319960 non-null  object 
 13  Unnamed: 13     625 non-null     object 
 14  Unnamed: 14     12 non-null      object 
 15  Unnamed: 15     4 non-null       object 
 16  Unnamed: 16     1 non-null       float64
dtypes: float64

<br>

Tentokrát soubor otevřeš pouze s varováním.

Pro příště je určitě výhodou vytvořit kopii takového souboru, kterou uložíš v ideálním kódování:

In [49]:
spravna_sada.to_csv("kopie_sada_utf8.csv", encoding="utf-8")

In [51]:
pokus = read_csv("vzorek_2.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [50]:
for line in open("vzorek_2.csv", 'rb'):
    detector.feed(line)
    if detector.done:
        break

detector.close()
print(detector.result)

{'encoding': 'Windows-1252', 'confidence': 0.7299788775910958, 'language': ''}


In [58]:
pokus_2 = read_csv("vzorek_2.csv", encoding="Windows-1252", delimiter=";")

In [60]:
pokus_2.head(6)

,ID,Jmeno,Prijmeni,Vek
0,123124.0,Matouš,Polek,24
1,123125.0,Marek,Lolek,33
2,123126.0,Petr,Dolek,90
3,123127.0,Filip,Vdolek,11
4,123128.0,Marcel,Trolek,27
5,123129.0,Lukáš,Colek,17


<br>

## Nekonzistentní data

---

Pokud ti data nechybí, ještě neznamená, že musí být nutně v pořádku:

In [64]:
from pandas import DataFrame

In [69]:
chybny_dataset = {
    "id": [111, 112, 113, 114, 115, 116],
    "jmeno": ["Matous", "Marek", "Petr", "Filip", "Jan", "Lukas"],
    "vek": [22, 29, 31, 55, 43, 61],
    "zeme": ["Ceska republika", "Slovensko", "Nemecko", "Ceskarepublika", "Ceska republika", "Rakouskou"]
}

In [70]:
df_zamestnanci = DataFrame(chybny_dataset)

In [71]:
df_zamestnanci.head()

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceskarepublika
4,115,Jan,43,Ceska republika


Různé zdroje, správci můžou způsobit nejednotný zápis a dopustit se *nekonzistence*.

In [68]:
df_zamestnanci["zeme"].unique()

array(['Ceska republika', 'Slovensko', 'Nemecko', 'ceskarepublika',
       'Rakouskou'], dtype=object)

<br>

To můžou být jak malá velká písmena, tak různé znaky, chybějící mezery apod.

Odstranit je není náročné.

Náročné může být opět rozpoznání, takové komplikace.

In [72]:
df_zamestnanci["zeme"].str.lower()

0    ceska republika
1          slovensko
2            nemecko
3     ceskarepublika
4    ceska republika
5          rakouskou
Name: zeme, dtype: object

<br>

Některé chyby, ale můžou způsobit paseku. Třeba chybějící mezery.

Tady je nejlepší, pomoci si vhodným nástrojem `fuzzywuzzy`.

In [73]:
!pip install fuzzywuzzy

In [77]:
import fuzzywuzzy
from fuzzywuzzy import process

/home/matous/projects/data-csob-2023/shared/onsite/env/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Pro menší datasety s chybějícími mezerami aj., může tato knihovna pracovat prakticky sama.

Účel této knihovny je rozpoznat podobné řetězce, jaké

In [78]:
shodne = fuzzywuzzy.process.extract(
    "Ceska republika",
    df_zamestnanci["zeme"],
    limit=5,
    scorer=fuzzywuzzy.fuzz.token_sort_ratio
)

In [79]:
shodne

[('Ceska republika', 100, 0),
 ('Ceska republika', 100, 4),
 ('Ceskarepublika', 97, 3),
 ('Slovensko', 25, 1),
 ('Rakouskou', 25, 5)]

Nyní můžeš vidět shodné `str` se zvoleným zadáním, jak to vidí `fuzzywuzzy`.

Pro nahrazení je optimální nachystat vhodnou uživatelskou funkci:

In [84]:
def nahrad_shody_stringem(dframe, sloupec, vzor, min_shoda=60):
    vsechny_stringy = dframe[sloupec].unique()
    
    shody = fuzzywuzzy.process.extract(vzor, vsechny_stringy, limit=5,
                                       scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    nejblizsi_shoda = [shoda [0] for shoda in shody if shoda[1] >= min_shoda]
    
    shodujici_zaznam = dframe[sloupec].isin(nejblizsi_shoda)
    dframe.loc[shodujici_zaznam, sloupec] = vzor

In [85]:
nahrad_shody_stringem(dframe=df_zamestnanci, sloupec="zeme", vzor="Ceska republika")

In [86]:
df_zamestnanci

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceska republika
4,115,Jan,43,Ceska republika
5,116,Lukas,61,Rakouskou


<br>

**🧠 CVIČENÍ 🧠, procvič si práci s chybným datasetem**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Nahraj soubor do `DataFrame`, jméno souboru `cviceni_3.csv` (správný encoding, správné rozdělení),
2. projdi sloupce, zkontroluj, kde je problém s daty,
3. vypiš všechny záznamy, které mají ve sloupci `mesto` hodnotu `Praha`.

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    df_ukazka = read_csv("cviceni_3.csv", encoding="Windows-1250", delimiter=";")

    def nahrad_shody_stringem(df, vzor, vyber, min_shoda=85):
        shody = fuzzywuzzy.process.extract(vzor, vyber, limit=20,
                                           scorer=fuzzywuzzy.fuzz.token_sort_ratio)

        nejblizsi_shoda = [shoda[0] for shoda in shody if shoda[1] >= min_shoda]

        shodujici_zaznam = vyber.isin(nejblizsi_shoda)
        df.loc[shodujici_zaznam, "mesto"] = vzor
        
    nahrad_shody_stringem(df_ukazka, "Praha", df_ukazka["mesto"])
    vystup = df_ukazka[df_ukazka["mesto"] == "Praha"]
    ```
</details>

---